<div style="float:left">
    <h1 style="width:450px">Practical 8: </h1>
    <h2 style="width:450px">Visualisation and Linked Data</h2>
</div>
<div style="float:right"><img width="100" src="https://github.com/jreades/i2p/raw/master/img/casa_logo.jpg" /></div>

<div style="border: dotted 1px rgb(156,121,26); padding: 10px; margin: 5px; background-color: rgb(255,236,184)"><i>Note</i>: You should download this notebook from GitHub and then save it to your own copy of the repository. I'd suggest adding it (<tt>git add Practial-...</tt>) right away and then committing (<tt>git commit -m "Some message"</tt>). Do this again at the end of the class and you'll have a record of everything you did, then you can <tt>git push</tt> it to GitHub.</div>

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns

import matplotlib.cm as cm
import matplotlib.pyplot as plt

## Task 1. Find London MSOAs

#### Grab E+W MSOA Shapefile

This -- and a lot of other data besides! -- can be accessed from the [GeoPortal](https://geoportal.statistics.gov.uk/datasets/middle-layer-super-output-areas-december-2011-ew-bgc-v2). And see also my discussion on [lookup tables](https://geoportal.statistics.gov.uk/datasets/postcode-to-output-area-to-lower-layer-super-output-area-to-middle-layer-super-output-area-to-local-authority-district-december-2011-lookup-in-england-and-wales).

In [ ]:
msoas = gpd.read_file('https://github.com/jreades/i2p/blob/master/data/src/Middle_Layer_Super_Output_Areas__December_2011__EW_BGC_V2-shp.zip?raw=true')
msoas.plot()

In [ ]:
msoas.sample(3)

#### Grab the Borough Boundaries

In [ ]:
boros = gpd.read_file('https://github.com/jreades/i2p/blob/master/data/src/Boroughs.gpkg?raw=true')

We can then generate the boundary for London using the `unary_union` operator (as we do here) or using the earlier [`dissolve(by=region)`](https://geopandas.org/aggregation_with_dissolve.html) approach. Consider the pros and cons of each approach in terms of performance, output format, and legibility. 

In [ ]:
ldn = gpd.GeoDataFrame(gpd.GeoSeries(data=boros.unary_union, crs='epsg:27700')).rename(columns={0:'original'})

# In order to ensure that we get all MSOAs _within_ London 
# we will buffer the boundary by 250m. If _cover_ were 
# easier to use then that option might be preferable.
ldn['geometry'] = ldn.original.buffer(250)
ldn = ldn.set_geometry('geometry')
ldn.plot();

#### Task 1.1: Select London MSOAs using a Spatial Join

Here's our first spatial join. By default it will be an _inner_ join.

In [ ]:
ldn_msoas = gpd.sjoin(msoas, ldn, op='??')
ldn_msoas.plot();

We no longer really need to keep the full MSOA data set hanging about.

In [ ]:
del(msoas)

#### Task 1.2: Extract Borough Names

Notice that the MSOA Name _contains_ the Borough name. Use a regex (in `str.replace()`) to extract the LA name from the MSOA name.

In [ ]:
ldn_msoas['Borough'] = ldn_msoas.MSOA11NM.str.replace(??,'',regex=True)

# Just check results look plausible; you should have:
# - 33 boroughs
# - A df shape of 983 x 14
print(ldn_msoas.Borough.unique())
print(len(ldn_msoas.Borough.unique()))
print(ldn_msoas.shape)

#### Task 1.3: Add MSOA Names using a Merge

The House of Commons Library provides a [MSOA Names](https://visual.parliament.uk/msoanames) data set that contains locally-relevant names applied to MSOAs. These seek to connect the Census geography (OA > LSOA > MSOA > LA) to a loosely-defined 'neighbourhood'.

In [ ]:
msoa_nms = pd.read_csv('https://github.com/jreades/i2p/blob/master/data/src/MSOA-Names-1.8.csv.gz?raw=true', compression='gzip')

In [ ]:
print(msoa_nms.columns.values)
msoa_nms.sample(3, random_state=42)

Now that you've loaded the `msoa_nms` data you need to merge it with our `ldn_msoas`. You will need to deal with the fact that the left and right fields have different names and may also want to think about the `how` of the merge.

In [ ]:
msoas = pd.merge(ldn_msoas, msoa_nms, ??, ??, ??)
print(msoas.shape)
print(type(msoas)) # You should check this -- result isn't always be a GeoDataFrame
msoas.sample(3, random_state=42)[['OBJECTID','MSOA11CD','MSOA11NM','msoa11hclnm']]

Your result should be:

|    | OBJECTID | MSOA11CD | MSOA11NM | msoa11hclnm |
| -: | -------: | -------: | :------- | :---------- |
| **810** | 811 | E02000841 | Sutton 002 | St Helier South |
| **801** | 802 | E02000832 | Southwark 026 | Nunhead North |
| **813** | 814 | E02000844 | Sutton 005 | The Wrythe |

#### Tidy Up

In [ ]:
to_drop = ['MSOA11NMW','LONG','LAT','Shape__Are','Shape__Len','index_right',
           'original','msoa11cd','msoa11nm','msoa11nmw','Laname','msoa11hclnmw']
msoas.drop(columns=to_drop, inplace=True)
print(msoas.shape)

#### And Save

In [ ]:
msoas.to_file(os.path.join('data','geo','London_MSOAs.gpkg'), driver='GPKG')

## Task 2. Load InsideAirbnb Data

Load the InsideAirbnb data into a DataFrame.

In [ ]:
cols = ['id','name','price','latitude','longitude','property_type','room_type']
df = pd.read_csv(??, usecols=cols, low_memory=False)

Clean the data:

In [ ]:
df['price'] = df.price.str.replace('$','').str.replace(',','').astype(float)
df.drop(df[((df.latitude.isna())|(df.longitude.isna()))].index, axis=0, inplace=True)
df.drop(df[((df.latitude < 40)|(df.longitude > 1))].index, axis=0, inplace=True)
print(df.shape)

#### Task 2.1: Add Geometry and Reproject

Convert the df into a GeoDataFrame named gdf:

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=??)
gdf = gdf.to_crs('epsg:27700')

#### Task 2.2: Add LA Names to Data

Associate LA (Local Authority) names to the listings using a spatial join:

In [ ]:
gdf_la = gpd.sjoin(gdf, boros, ??, ??)
print(gdf_la.columns.values)

In [ ]:
print(gdf_la.shape)
gdf_la.sample(3, random_state=42)[['id','name','NAME']]

You should have a GeoDataFrame with 74,184 rows and 15 columns and the following tabular output:

|       | id | name | NAME |
| ----: | -------: | :---------- | :---------- |
| 66121 | 41808059 | Haverstock Hotel, Twin Room | Camden |
| 58701 | 38685213 | Charming 1st floor two bedroom Fulham apartment| Hammersmith and Fulham |
| 53195 | 35724891 | Lovely East End En-Suite Room in Great Location. | Tower Hamlets |

#### Task 2.3: Tidy Up

In [ ]:
gdf_la.drop(columns=['index_right','HECTARES','NONLD_AREA','ONS_INNER'], inplace=True)
gdf_la.NAME.unique()

There's something strange about the output of `unique()` above... can you spot it? There are 15 problematic records... see if you can print them out here:

In [ ]:
gdf[??][['id','name']]

#### Task 2.4: Map Problematic Listings

Now see if you can map them as below:

In [ ]:
ax = ??.plot()
boros.plot(ax=ax, edgecolor='r', facecolor='None', alpha=0.5)

Your result should be:

![](https://github.com/jreades/i2p/blob/master/lectures/img/Unmatched_Listings.png?raw=true)

#### Task 2.5: Drop Problematic Results

In [ ]:
gdf_la.drop(index=??, axis=1, inplace=True)
print(gdf_la.shape)

You should now have 74,169 records.

#### Task 2.6: Check Results

In [ ]:
ax = gdf_la.plot(column='NAME', markersize=0.5, alpha=0.5, figsize=(10,8))
boros.plot(ax=ax, edgecolor='r', facecolor='None', alpha=0.5);

## Task 3. Using Seaborn and Grouped Data

#### Task 3.1: A Basic Boxplot

See if you can create a basic boxplot of price by room type, and then set the y-limits to 0-500 (£/night).

In [ ]:
sns.set_style('whitegrid')
sns.set_theme(style="ticks", palette="pastel")
f,ax = plt.subplots(figsize=(8,5))
??
sns.despine(offset=10)
ax.??([0,500])

#### Task 3.2: Grouping on One Column

Let's group the data on the LA name. You should get that the type is a `DataFrameGroupBy`.

In [ ]:
la_data = gdf_la.groupby(??)
print(type(la_data))

Notice the result here:

In [ ]:
la_data.price.mean()

#### Task 3.3: A Basic Distribution Plot

Although this is hardly the most robust analysis (subtracting the mean of a grouped mean from a grouped mean seems... sketchy), it's useful for quickly demonstrating a different plot type and then asking you to work out how to obtain something close to the output below using these solutions:

- [Figure title](https://matplotlib.org/3.3.3/gallery/subplots_axes_and_figures/figure_title.html)
- [How to position `suptitle`](https://stackoverflow.com/questions/55767312/how-to-position-suptitle)
- [Label axes on Seaborn Barplot](https://stackoverflow.com/questions/31632637/label-axes-on-seaborn-barplot)

In [ ]:
sns.set_style('darkgrid')
ax = sns.displot((la_data.price.mean() - la_data.price.mean().mean())/la_data.price.std())
??

Your result should be close to:

![](https://github.com/jreades/i2p/blob/master/lectures/img/std_boro_price.png?raw=true)

#### Task 3.4: Grouping on Multiple Columns

We're now going to create a data frame grouped by LA and room type, and then use the `agg` (aggregate) function on a grouped data frame and derive several measures (Count, Sum, Mean, Median, std, Lower Quartile, and Upper Quartile) in one go from the price column. 

You will want to search on something like "Multiple aggregations of the same column using pandas GroupBy.agg()" in order to work this out. 

You'll also need to look at "calculate quantile pandas aggregate" -- depending on the solution you choose, your quantiles may or may not have the same labels as mine below.

In [ ]:
la_data2 = gdf_la.groupby(??).agg(??).reset_index()

In [ ]:
la_data2.sample(3, random_state=42)

My sample gives me:

|    | NAME | room_type |     |     |     |     |     |     | price |
| -: | :--- | :-------- | --: | --: | --: | --: | --: | --: | ----: |
|    |      |           | **count** | **sum** | **mean** | **median** | **std** | **\<lambda_0\>** | **\<lambda_1\>** |
| 18 | Bromley | Private room | 334 | 17288.86 | 51.763054 | 36.0 | 91.390795 | 27.0 | 50.00 |
| 42 | Greenwich | Private room | 830 | 62005.70 | 74.705663 | 38.0 | 223.600981 | 28.0 | 53.75 |
| 36 | Enfield | Entire home/apt | 217 | 26437.72 | 121.832811 | 95.0 | 121.763967 | 70.0 | 130.00 |

I am _not_ cruel enough to make you work out how to convert the above to a flat index, so here's the code to do that:

In [ ]:
la_data2.columns = la_data2.columns.droplevel(0)
print(la_data2.columns.values)

So now we just need to correct a few column labels manually:

In [ ]:
cols = la_data2.columns.values
cols[ 0] = 'name'
cols[ 1] = 'room_type'
cols[-2] = 'lq'
cols[-1] = 'uq'
la_data2.columns = cols

This next plot may take a while to run:

In [ ]:
sns.set_theme(style="whitegrid")

f, ax = plt.subplots()
sns.despine(bottom=True, left=True)

# Show each observation with a scatterplot
# https://seaborn.pydata.org/examples/jitter_stripplot.html
g = sns.stripplot(x="median", y="room_type", hue="name",
              data=la_data2, dodge=True, alpha=.25, zorder=1)
g.legend_.remove()

# Show the median of medians
sns.pointplot(x="median", y="room_type",
              data=la_data2.groupby('room_type').agg('median').reset_index(), 
              dodge=.232, join=False, palette="dark",
              markers="d", scale=.75, ci=None)

plt.gcf().subplots_adjust(top=0.92)
f.suptitle('Median Price by Borough', fontsize=14);

#### Task 3.5: Grouping on a Custom Mapping

Use the map below, taken from the Centre for London's [_Housing and Inequality in London_](https://www.centreforlondon.org/wp-content/uploads/2016/08/CFLJ4292-London-Inequality-04_16_WEB_V4.pdf) (p.17) to assign each LA to one of the five regions shown. Create a grouped data frame using this mapping instead of the LA one. Do so _without_ creating a new column.

![](https://github.com/jreades/i2p/blob/master/lectures/img/Borough_Mapping.png?raw=true)

Set up a `mapping` dict here so that you can apply it as part of the `groupby` operation below (you should have 33 keys when done):

In [ ]:
mapping = {}
#I've given you the first region to start
for b in ['Enfield','Waltham Forest','Redbridge','Barking and Dagenham','Havering','Greenwich','Bexley']:
mapping[b]='Outer East and North East'
??
print(len(mapping.keys()))

Group by your mapping and by `room_type`, and calculate the mean and median price only:

In [ ]:
la_data3 = gdf_la.set_index('NAME').groupby([mapping,'room_type']).agg(
    {'price':['mean','median']}
).reset_index()

In [ ]:
la_data3.sample(3, random_state=42)

My output looks like this:

|     | level_0 | room_type |     | price |
| --: | :------ | :-------- | --: | ----: |
|     |         |           | **mean** | **median** |
| 0 | Inner East | Entire home/apt | 138.428470 | 100.0 |
| 17 | Outer West and North West | Hotel room | 123.295429 | 85.0 |
| 15 | Outer South | Shared room | 57.0000000 | 45.0 |

In [ ]:
la_data3.columns = la_data3.columns.droplevel(0)
cols = la_data3.columns.values
cols[0]  = 'area'
cols[1]  = 'room_type'
la_data3.columns = cols

In [ ]:
la_data3.head(3)

See if you can produce the same plot as above but for the aggregated areas:

In [ ]:
sns.set_theme(style="whitegrid")


## Task 4. Exploring an LA using Subplots

Select a LA that is relevant to _you_ to explore further...

In [ ]:
LA = 'Waltham Forest'

#### Task 4.1: Spatial Join Listings and MSOAs

The first thing we want to do is join MSOA identifiers in `msoas` to each listing in `gdf_la`. In both cases we want to constrain the data to only be for 'our' LA of interest. You do not have to save this to a new variable and can do the selection _as part_ of the spatial join:

In [ ]:
msoadf  = gpd.sjoin(
            gdf_la[??], 
            ??, 
            op=??)

#### Task 4.2: Aggregate by MSOA

Now aggregate the data by MSOA, deriving median price and a count of the listings:

In [ ]:
msoagrdf = msoadf.groupby('MSOA11NM').agg({'price':['median','count']}).reset_index()

Which level value is easier to use? 0? or 1?

In [ ]:
msoagrdf.columns = msoagrdf.columns.get_level_values(1)
msoagrdf.head()

Fix the missing column name:

In [ ]:
msoagrdf.rename(columns={'':'MSOA11NM', 'count':'listings'}, inplace=True)

#### Task 4.3: Join on Indexes

Here we see the difference between merge and join. You'll notice that `join` operates taking one data frame as the implicit 'left' table (the one which _calls_ join) while the one that is passed to the join function is, implicitly, the 'right' table. Join operates only using indexes, so you'll need to insert the code to specify the same index on both data frames, but this can be done **on-the-fly** as part of the joining operation:

In [ ]:
msoa_gdf = msoagrdf.set_index(??).join(
                ??, 
                rsuffix='_r')
msoa_gdf.head(3)

You need to add a command in order to help python recognise that this should be a GeoDataFrame:

In [ ]:
msoa_gdf.plot(column='median', legend=True)

#### Task 4.4: Discover Fonts Installed

I find matplotlib's use of fonts to be _profoundly_ weird. Basically, you need to investigate the 'font cache' folder and see what's listed there. Depending on how you installed the Python libraries you _might_ have access to your whole computer's fonts (if you used `conda`) or to only those free fonts that come with Linux (if you used Vagrant or Docker). This code may not even run on a PC or Mac with `conda` in which case you'll need to do some more investigating and poking around...

In [ ]:
import matplotlib
loc = matplotlib.get_cachedir()
!ls {loc}

Hopefully you will see a list of installed fonts when you run this. See if you can make sense of what this code does!

In [ ]:
fonts = !cat {loc + '/fontlist-v330.json'}
fonts = set(list(filter(lambda x:'"name"' in x, fonts)))
fonts = [x.replace('      "name": "','').replace('",','') for x in fonts]
print(fonts)

I am setting this as the 'title font' (`tfont`) to use in the output below. You can pick another font and see what happens. The format for this is a dictionary, so where you see `fontdict` in the `matplotlib` documentation this should work:

In [ ]:
tfont = {'fontname':'Liberation Sans Narrow', 'horizontalalignment':'left'}

#### 4.5: Create at Atlas-Type Output

We're now going to emulate a _bit_ of QGIS' Atlas function by creating two subplots and then adding a _third_ plot afterwards that shows where the borough is.

In [ ]:
f,axes = plt.subplots(1,2,figsize=(12,8))

msoa_gdf.plot(column='median', ax=axes[0], legend=True, cmap='viridis')
msoa_gdf.copy().plot(column='listings', ax=axes[1], legend=True, cmap='plasma')
for ax in axes:
    ax.axis('off')
    
ax2 = f.add_axes([0.015, 0.7, 0.2, 0.2])
boros.plot(facecolor='lightgrey', edgecolor='None', ax=ax2)
boros[boros.NAME==LA].plot(facecolor='r', edgecolor='r', hatch='///', ax=ax2)
ax2.axis('off')

f.suptitle(LA, x=0.025, ha='left', size=24, **tfont)
axes[0].set_title('Median Price', size=20, **tfont)
axes[1].set_title('Listings', size=20, **tfont)

plt.figtext(x=0.025, y=0.65, s=f"Total Listings: {msoa_gdf.listings.sum():,.0f}", size=14, **tfont);

#### Bonus Achievement Unlocked!

See if you can convert the above to an _actual_ atlas output: you'll want to turn this into a function so as to be able to produce (and save) the map for _every_ borough. You'll even need to parameterise the filename so that you save to _different_ PNG files...

## Task 5. Using Bokeh

#### Task 5.1: For a Chart

Group the listings by Borough and Room Type, and aggregate by median price, also producing a count variable for the number of listings of each type in each Borough.

In [ ]:
la_rt = gdf_la.groupby(??).agg(??).reset_index()
la_rt.columns = ['NAME','room_type','price','listings']
la_rt.head()

Choose a LA of interest:

In [ ]:
LA = 'Camden'

In [ ]:
la_rt[la_rt.NAME==LA]

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Spectral4
from bokeh.models import CustomJS, Dropdown

output_notebook()

room_types = list(la_rt[la_rt.NAME==LA].room_type.values)
price = list(la_rt[la_rt.NAME==LA].price.values)
boros = list(la_rt[la_rt.NAME==LA].NAME.values)
counts = list(la_rt[la_rt.NAME==LA].listings.values)

source = ColumnDataSource(data=dict(
                                room_types=room_types, 
                                price=price, 
                                counts=counts, 
                                boros=boros, 
                                color=Spectral4))

p = figure(x_range=room_types, plot_height=300, title=f"Median Price by Room Type in {LA}",
           toolbar_location=None, tools="")

p.vbar(x='room_types', top='price', width=0.9, color='color', legend_field="room_types", source=source)

p.xgrid.grid_line_color = None
p.y_range.start = 0

# Add hover
p.add_tools(
    HoverTool(tooltips = [("Borough", "@boros"),
                          ("Listings", "@counts"),
                          ("Median Price", "$@price/night")]
             )
)

show(p)

#### Task 5.2: For a Map

This is not the prettiest code, but it should work...

In [ ]:
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON, get_provider

from bokeh.io import output_file, show, output_notebook, push_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.plotting import figure
from bokeh.palettes import brewer

output_notebook()

I cobbled the mapping functions below together from two tutorials I found online ([this one](https://github.com/dmnfarrell/teaching/blob/master/geo/maps_python.ipynb) and [this one](https://widdowquinn.github.io/Teaching-Data-Visualisation/exercises/interactive_bokeh_map/interactive_bokeh_map.html)). As you can see, this is a very different approach to mapping data, but it has clear benefits for exploratory purposes and produces fast, interactive maps... and I've not even added selection and filtering tools!

In [ ]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)

def bokeh_plot_map(gdf, column=None, title=''):
    """Plot bokeh map from GeoJSONDataSource """
    
    tile_provider = get_provider(CARTODBPOSITRON)

    geosource = get_geodatasource(gdf)
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    vals = gdf[column]
    
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette=palette, low=vals.min(), high=vals.max())
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=10,
                         location=(0,0), orientation='horizontal')

    tools = 'wheel_zoom,pan,reset,hover'
    
    p = figure(title = title, plot_height=700, plot_width=850, toolbar_location='right', tools=tools)
    p.add_tile(tile_provider)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    # Add patch renderer to figure
    p.patches('xs','ys', source=geosource, fill_alpha=0.5, line_width=0.5, line_color='white',  
              fill_color={'field' :column , 'transform': color_mapper})
    
    # Specify figure layout.
    p.add_layout(color_bar, 'below')
    
    # Add hover
    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [("Borough", "@Borough"),
                      ("Name", "@msoa11hclnm"),
                      ("Listings", "@listings"),
                      ("Median Price", "£@price")]
    
    return p

First, spatially join individual listings to MSOAs. You will need to think about the `how` here!

In [ ]:
msoadf  = gpd.sjoin(??, ??, how=??, op=??)
msoadf.sample(3, random_state=42)

Then group by and aggregate based on median price and count. Notice how the output from this differs subtly from what you'd get with `{'price':['median','count']}`!

In [ ]:
msoagrdf2 = msoadf.groupby('MSOA11NM').agg({'price':['median'],'room_type':['count']}).reset_index()
msoagrdf2.head()

Reset the levels and fix the column names:

In [ ]:
msoagrdf2.columns = msoagrdf2.columns.get_level_values(0)
msoagrdf2.rename(columns={'room_type':'listings'}, inplace=True)
msoagrdf2.head()

Check the data range(s) and resolve any apparent issues:

In [ ]:
msoagrdf2.sort_values(by='price', ascending=False).head(5)

In [ ]:
msoagrdf2.loc[??,??] = np.nan

Check the results of any modification:

In [ ]:
msoagrdf2.sort_values(by='price', ascending=False).head(5)

Read in the MSOA GPKG file that we saved earlier:

In [ ]:
msoas = gpd.read_file(os.path.join('data','geo','London_MSOAs.gpkg'))
msoageo = pd.merge(msoas, msoagrdf2, on='MSOA11NM')

Reproject to Web Mercator:

In [ ]:
msoageo = msoageo.to_crs('epsg:3785')
msoageo.total_bounds

And map it!

In [ ]:
p = bokeh_plot_map(msoageo, 'price', title='MSOA-Level Activity')

handle = show(p, notebook_handle=True)
push_notebook(handle=handle)